# 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as mp
import math
from sklearn import preprocessing
from scipy.io import arff


In [2]:
data =  arff.loadarff('CM1.arff')
df = pd.DataFrame(data[0])
data = np.array(df)
m,n=data.shape

In [3]:
#compute k
list_NY = []
list_data = list(data[:,-1])
N = list_data.count(b'N')
Y = list_data.count(b'Y')
print(N,Y)
k = math.floor(N/Y)

302 42


In [4]:
# compute scale
data_scaled=preprocessing.scale(data[:,:-1], axis=0, with_mean=True,with_std=True,copy=True) 

data_scaled_NY = np.concatenate((data_scaled,data[:,-1].reshape(m,1)),axis=1)

index = np.arange(m)

data_scaled = np.concatenate((index.reshape(m,1),data_scaled_NY),axis=1)

# print(data_scaled)
data_scaled_N_index  = list(np.argwhere(data_scaled_NY[:,-1]==b'N'))
data_scaled_Y_index  = list(np.argwhere(data_scaled_NY[:,-1]==b'Y'))

data_scaled_N = data_scaled[data_scaled_N_index].reshape(N,n+1)
data_scaled_Y = data_scaled[data_scaled_Y_index].reshape(Y,n+1)
print(data_scaled_N)
print(data_scaled_Y)

[[0 -0.44152387584377945 -0.21591477298003958 ... -1.3207150373410579
  -0.362232068967316 b'N']
 [3 -0.39760131560946477 -0.5791161514892503 ... -1.5318055891490587
  -0.5974777576722008 b'N']
 [4 1.5349913347003807 0.7526222363778557 ... -0.9159489042492164
  1.1035295298861971 b'N']
 ...
 [341 -0.22191107467220608 -0.21591477298003958 ... -1.072683638966657
  -0.05460309143015892 b'N']
 [342 -0.6611366770153528 -0.5791161514892503 ... -1.5318055891490587
  -0.651765224296405 b'N']
 [343 -0.26583363490652073 0.026219479359434257 ... 0.07037169907366737
  -0.09079473584629505 b'N']]
[[1 -0.04622083373494737 -0.3369818991497765 ... 0.5379372713283891
  -0.23556131351083956 b'Y']
 [2 0.48084988907682874 -0.21591477298003958 ... 0.962757006841991
  -0.21746549130277149 b'Y']
 [32 6.102937599069108 3.537166138281805 ... 1.023445540486791
  2.9854950395252757 b'Y']
 ...
 [328 0.8322303709513461 0.5104879840383819 ... 0.8603780892151103
  0.8320921967651762 b'Y']
 [329 -0.5293689963124089 -

In [5]:
NY_distance_samples_neighbor = np.empty([0,3],float)
flag = 0
i=0
for s_Y in data_scaled_Y:
    temp_arr = np.array([])
    for s_N in data_scaled_N:
        temp = np.linalg.norm(s_N[1:n]-s_Y[1:n])
        temp_arr = np.append(temp_arr,temp)

    temp_arr = temp_arr.reshape((N,1))
    NY_distance = np.concatenate((data_scaled_Y[[i],[0]].repeat(N).reshape(N,1),data_scaled_N[:,[0]],temp_arr),axis=1)
    NY_distance_sample_neighbor = NY_distance[NY_distance[:,2].argsort()].reshape(N,3)[0:k,:]
    NY_distance_samples_neighbor = np.append(NY_distance_samples_neighbor,NY_distance_sample_neighbor,axis=0)
    i = i+1
NY_M,NY_N = NY_distance_samples_neighbor.shape
print(NY_distance_samples_neighbor)


[[1 77 1.3864736038514105]
 [1 158 1.5422582477056133]
 [1 101 1.727529267366282]
 [1 27 1.7421482651592894]
 [1 276 1.750944307795286]
 [1 159 1.770417880239988]
 [1 12 1.7809599915585421]
 [2 276 1.5940683250900927]
 [2 77 1.6418133165087703]
 [2 13 1.7443691138147748]
 [2 243 1.897467867974438]
 [2 235 1.925041249459487]
 [2 171 1.9857724716445653]
 [2 269 1.992651229360749]
 [32 236 9.81996026030024]
 [32 283 10.332319529496163]
 [32 181 10.616074515351448]
 [32 282 11.254175729424928]
 [32 83 11.44572567296318]
 [32 117 11.50616388771017]
 [32 190 11.930795868184584]
 [34 284 3.070398468487511]
 [34 20 3.0838191125359047]
 [34 267 3.1524376610566422]
 [34 36 3.336610239641819]
 [34 31 3.4432623192817604]
 [34 297 3.4513033361446896]
 [34 219 3.5342676351978226]
 [35 110 0.9049934770139791]
 [35 148 1.3277615557609224]
 [35 238 1.328369897199379]
 [35 40 1.441778541774755]
 [35 273 1.4678701598334236]
 [35 209 1.5156988768460664]
 [35 157 1.5194323646631307]
 [39 46 2.5484139549866

In [6]:
#cal feature differential
w_matrix = np.zeros(n-1).reshape(1,n-1)

index_ny = NY_distance_samples_neighbor[0:NY_M,0:NY_N-1]

index_y = index_ny[0:NY_M,0].reshape(NY_M).astype(int)
index_n = index_ny[0:NY_M,1].reshape(NY_M).astype(int)

index_last = np.arange(m)
data_filter = np.concatenate((index_last.reshape(m,1),data.copy()),axis=1)

samples_feature_differential = np.abs(data_filter[list(index_y),1:n]-data_filter[list(index_n),1:n])

feature_index = np.arange(n-1).reshape(1,n-1)
for feature_differential in samples_feature_differential:
    
    temp_feature = np.concatenate((feature_index,feature_differential.reshape(1,n-1)),axis=0)
    temp_sorted_feature = temp_feature[:,temp_feature[1].argsort()]
#     print(temp_sorted_feature) 
    feature = temp_sorted_feature[1,:].copy()

    j = 1 
    temp_w = np.empty(n-1)
    temp_w[0] = j
    for i in range(n-1):
        if i!=0 :
            if  feature[i-1]==feature[i]:
                temp_w[i] = j
            else:
                j = j+1
                temp_w[i] = j
    temp_sorted_feature[1,:]=temp_w
#     print(temp_sorted_feature)
    temp_last_feature  = temp_sorted_feature[:,temp_sorted_feature[0].argsort()]
#     print(temp_last_feature)
    w_matrix = np.add(temp_last_feature[1,:],w_matrix)
    
index = np.arange(n-1).reshape(1,n-1)
last = np.concatenate((index,w_matrix),axis=0)
last = last[:,last[1].argsort()][::,::-1]
print(last)

[[19 23 24 21 31 34 30 17 4 36 15 32 0 35 12 28 18 5 33 3 1 27 8 6 2 26
  10 16 20 13 25 11 14 7 9 29 22]
 [7367.0 6938.0 6844.0 6056.0 5572.0 5275.0 5110.0 4954.0 4527.0 4523.0
  4408.0 4281.0 4209.0 3927.0 3808.0 3617.0 3483.0 3402.0 3100.0 2962.0
  2743.0 2736.0 2563.0 2348.0 2334.0 2211.0 2197.0 1835.0 1510.0 1429.0
  1268.0 1218.0 987.0 913.0 912.0 756.0 631.0]]


In [7]:
# ex2
ex2_array = data_filter[:,1:-1]
sample_number,feature_number = ex2_array.shape

#离散化
types=100
Features_Classes = np.empty(sample_number*feature_number).reshape(sample_number,feature_number)
for i in range(feature_number):
    temp = list(ex2_array[:,i])
    bins = list(np.linspace(min(temp),max(temp),types))
    classes = pd.cut(temp,bins)
    Features_Classes[:,i] = classes.codes
print(Features_Classes)


def P_H(x,sample_number):
    #输入一个离散化特征，输出H值
    X = np.unique(x)
    p = []
    for j in X:
        temp=np.sum(x==j)
        p.append(temp/sample_number)
    p = np.array(p)
#     print(p)
    H_x = -np.sum(np.multiply(p,np.log2(p)))
#     print(H_x)
    return H_x


def H_XY(x,y,sample_number):
#输入两个特征，输出两个特征下的条件熵
    Y = np.unique(y)
    p = []
    for j in Y:
        temp=np.sum(y==j)
        p.append(temp/sample_number)
    p = np.array(p)
    Con_xy = np.concatenate((x.reshape(sample_number,1),y.reshape(sample_number,1)),axis=1)
    H_XY_ = 0
    j=0
    for i in Y:
        #每个Y取定为y的概率,并生成对应的xy矩阵，再应用条件概率公式
        index_list_xy = np.argwhere(i==Con_xy[:,1])
        temp_xy_matrix = Con_xy[index_list_xy]#取到xy矩阵
#         print(temp_xy_matrix)
        temp_xy_matrix = temp_xy_matrix.reshape(-1,2)
#         print(temp_xy_matrix.shape)
        m_xy = temp_xy_matrix.shape
        #按照定义计算
        H_XY_ = np.add(H_XY_,-p[j]*(-P_H(temp_xy_matrix[:,0],m_xy[0])))
        j = j+1
        # print(H_XY_)
    return H_XY_

def IG_XY(x,y,sample_number):
    H_x = P_H(x,sample_number)
    H_xy = H_XY(x,y,sample_number)
    IG_XY_ = H_x-H_xy
    return IG_XY_

def SU_XY(x,y,sample_number):
    IG_XY_ = IG_XY(x,y,sample_number)
    H_x = P_H(x,sample_number)
    H_y = P_H(y,sample_number)
    SU_XY_ = 2*IG_XY_/(H_x+H_y)
    return SU_XY_

feature_su_matrix = np.zeros(feature_number*feature_number).reshape(feature_number,feature_number)
for i in range(feature_number):
    for j in range(feature_number):
        temp_SU_XY = SU_XY(Features_Classes[:,i],Features_Classes[:,j],sample_number)
#         print(temp_SU_XY)
#         print(type(temp_SU_XY))
        feature_su_matrix[i,j] = temp_SU_XY
print(feature_su_matrix)

[[ 3.  3.  7. ...  3.  5.  3.]
 [ 9.  2. 11. ...  7. 53.  4.]
 [16.  3.  3. ...  9. 65.  5.]
 ...
 [ 6.  3. 22. ...  6. 11.  6.]
 [ 0. -1. -1. ...  0. -1.  0.]
 [ 6.  6. 22. ...  7. 41.  6.]]
[[1.         0.35467926 0.32388281 ... 0.47869126 0.51775654 0.40834954]
 [0.35467926 1.         0.28874335 ... 0.42482582 0.40165344 0.45776577]
 [0.32388281 0.28874335 1.         ... 0.36329445 0.39136435 0.33682311]
 ...
 [0.47869126 0.42482582 0.36329445 ... 1.         0.51562545 0.53803536]
 [0.51775654 0.40165344 0.39136435 ... 0.51562545 1.         0.47704249]
 [0.40834954 0.45776577 0.33682311 ... 0.53803536 0.47704249 1.        ]]


In [41]:
a = np.array([1,4,3,1,3,3])
print(np.squeeze(np.argwhere(a==1)))

[0 3]
